In [1]:
import numpy as np

In [2]:
def twostock_ELS(r, T, vol, security_price, s, dividend,exercise_return, exercise_condition, barrier,correlation, simul_num):
    K = exercise_condition
    dt = 1/252
    t = 252*T
    np.random.seed(0)
    
    num = [0,0,0,0,0,0,0] #각 경우의 수가 담길 list 이후 이를 통해 각 경우의 확률을 구함
    payoff = [security_price * (1+i*exercise_return/2) for i in range (6)] #payoff가 knock in을 제외하고는 정해져 있으므로 list로 만듦
    discount = [np.exp(-r*i*T/6) for i in range (1,7)]
    
    loss_ratio = []
    
    ki_payoff = []
    s10 = s[0]
    s20 = s[1]
    rho = correlation
    d = dividend
    for i in range (simul_num):
        s1 = s10
        s2 = s20
        s1_path = [s1]
        s2_path = [s2]

        for j in range (t):
            u = np.random.standard_normal()
            v = np.random.standard_normal()
            ep1 = u
            ep2 = rho*u + np.sqrt(1-rho**2)*v
            s1 = s1*np.exp(r*dt+vol[0]*ep1*np.sqrt(dt))
            s1_path.append(s1)
            s2 = s2*np.exp(r*dt+vol[1]*ep2*np.sqrt(dt))
            s2_path.append(s2)
            
        if s1_path[int(t/6)] >= s10 * K[0] and s2_path[int(t/6)] >= s20 * K[0]:
            num[0] += 1
        elif s1_path[int(t/3)] >= s10 * K[1] and s2_path[int(t/3)] >= s20 * K[1]:
            num[1] += 1
        elif s1_path[int(t/2)] >= s10 * K[2] and s2_path[int(t/2)] >= s20 * K[2]:
            num[2] += 1
        elif s1_path[int(t*2/3)] >= s10 * K[3] and s2_path[int(t*2/3)] >= s20 * K[3]:
            num[3] += 1
        elif s1_path[int(t*5/6)] >= s10 * K[4] and s2_path[int(t*5/6)] >= s20 * K[4]:
            num[4] += 1
        elif s1_path[t] < s10 * K[5] or s2_path[t] < s20 * K[5]: # 70% 미만인 경우
            if min(s1_path) < s10 * barrier or min(s2_path) < s20 * barrier: #knock in 됐을 때
                num[6] += 1
                loss = [s1/s10, s2/s20]

                ki_payoff.append(min(loss)*security_price) #둘 중 손실률이 더 높은 걸 채택해서 증권가액에 곱해 payoff로 취급
                
            elif min(s1_path) >= s10 * barrier and min(s2_path) >= s20 * barrier:# knock in은 아닐 때
                num[5] += 1
        elif s1_path[t] >= s10 * K[5] and s2_path[t] >= s20 * K[5]:
            num[5] += 1
    
    ki_payoff = np.array(ki_payoff)
    payoff.append(ki_payoff.mean()) # 한 번에 계산하기 위해 payoff 리스트에 knock in payoff 넣음
    each_pr = [num[i]/simul_num for i in range(7)] # 각 조건의 확률을 가진 리스트
    temp = [discount[i] * each_pr[i] * payoff[i] for i in range (6)] # 할인, 확률, payoff가 들어간 리스트
    temp.append(discount[5] * each_pr[6] * payoff[6])
    ELS_price = sum(temp) # 합하면 기댓값
    
    return ELS_price
            
    

In [11]:
interest = np.log(1.037) # 1년물 이자율, 상수
T = 3
vol =[0.1795866657, 0.3391402795] # 각 6개월 변동성. 블랙숄즈 가정이므로 고정
security = 10000 #증권가액
s = [328.22, 2510.06] # 5월 9일자 기초자산의 가격
dividend = 0
exercise_return = 0.11 #연 11.00%
exercise_condition = [0.92, 0.9, 0.85,0.8,0.75,0.7]
barrier = 0.55
correlation = 0.5174259669 #기초자산 간 상관계수
simul_num = 4000



In [12]:
ELS_price = twostock_ELS(interest, T, vol, security, s, dividend, exercise_return, exercise_condition, barrier, correlation, simul_num)
print(ELS_price)

9546.458499515407
